In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
import pickle


In [ ]:
well_list = pd.read_csv("./locations.csv",sep=';',header = 0,encoding = 'Latin1')

for loc in range(well_list.shape[0]):
   
    ID = well_list.alias[loc]
    print(ID+": "+str(loc))
    
    method = 'lrp.z'
    file = './analysis_'+ID+'_'+method.replace('.','')+'.pickle'

    with open(file, 'rb') as f:
        imports = pickle.load(f)
    
    GLOBAL_SETTINGS = {
            'opt_start': pd.to_datetime('01012015', format='%d%m%Y'), 
            'test_start': pd.to_datetime('01012017', format='%d%m%Y'),
            'test_end': pd.to_datetime('31122020', format='%d%m%Y')
        }
    seqlength = 52
    y_list = imports['simulations']
    sim_members = np.zeros(shape = (y_list[0].shape[0],20))
    for i in range(20):
        sim_members[:,i] = y_list[i].reshape(-1,)
    sim_mean = np.nanmean(sim_members,axis = 1)
    
    data_investigate = imports['data_investigate']
    data=data_investigate
    obs = data_investigate['GWL']
    sim = sim_mean
    err = sim-obs
    err_rel = (sim-obs)/(np.max(data['GWL'])-np.min(data['GWL']))
    err_nash = obs - np.mean(np.asarray(data['GWL'][(data.index < GLOBAL_SETTINGS["test_start"])]))
    
    NSE = 1 - ((np.sum(err ** 2)) / (np.sum((err_nash) ** 2)))
    try:
        r = stats.pearsonr(sim[:,], obs[:,])
        r = r[0] #r
    except:
        r = [np.nan, np.nan]
        r = r[0] #r
    R2 = r ** 2
    alpha = np.std(sim)/np.std(obs)
    beta = np.mean(sim)/np.mean(obs)
    KGE = 1-np.sqrt((r-1)**2+(alpha-1)**2+(beta-1)**2) #KGE
    
    scores = pd.DataFrame(np.array([[NSE, KGE, R2]]),
                   columns=['NSE', 'KGE','R2'])
    scores.to_csv('./scores_full_timeseries_'+ID+'.txt',float_format='%.3f',index=False)
           
